이전 과정과 달리 max_depth으로 하면 성능이 달라지나요 ??
-> 아마 달라질 듯, 모든 값을 다 시도할 수 는 없지만 시간이 허락하는 대로 테스트하려 한다

이런저런 값으로 모델을 많이 만들어서 테스트 세트로 평가하면 결국 테스트 세트에 잘 맞는 모델이 만들어지는 것 아닌가? <br>
그러나 테스트 세트를 사용해 자꾸 성능을 확인하다 보면 점점 테스트 세트에 맞추게 된다

테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다 <br>
가능한 한 딱 한번만 사용하는 것이 좋다.

그렇다면 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝을 어떻게 할 수 있을까? <br>
게다가 결정 트리는 테스트해 볼 매개변수가 많다.

### 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. <br>
테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 **훈련 세트를 또 나누는 것**이다.

이 방법은 너무 단순해서 이상할 수도 있지만, 실제로 많이 사용하는 방법이다. <br>
전체 데이터 중 20%를 테스트 세트로 만들고 나머지 80%를 훈련 세트로 만들었다.

이 훈련 세트 중에서 다시 20%를 떼어 검증 세트로 만든다.

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다. <br>
이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다.<br>


그 다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델 다시 훈련한다. <br>
마지막에 테스트 세트에서 최종 점수를 평가한다. 아마도 실전에 투입했을 때 테스트 점수와 비슷한 성능을 기대할 수 있을 것이다.

이전 절에서 사용했던 데이터를 다시 불러와 검증 세트를 만들어 보겠다. 먼저 판다스로 csv 데이터를 읽는다.

In [2]:
import pandas as pd 
wine = pd.read_csv("https://bit.ly/wine_csv_data")

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

이제 훈련 세트와 테스트 세트를 나눌 차례이다. <br>
훈련 세트의 입력 데이터와 타깃 데이터를 train_input, train_target으로 나눈다.

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

그 다음 다시 train_input, train_target을 다시 train_test_split에 넣어 훈련 세트 <br>
sub_input, sub_target, 검증 세트 val_input, val_target을 만든다. 여기에서도 test_size = 0.2로 지정하여 <br>
train_input의 약 20%를 val_input으로 만든다.

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

단순히 두번 적용해서 세 단위의 세트를 만들어 낸다. <br>
훈련 세트와 테스트 세트의 크기를 확인해 보자.

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


원래 5197개였던 훈련 세트가 4157개로 줄고, 검증 세트는 1040개가 되었다.

이제 sub_input, sub_target와 val_input, val_target 사용해 모델을 만들고 평가해 보겠다.

In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


0.9971133028626413
0.864423076923077


이렇게 val_input, val_target 평가하면 된다. 이 모델은 확실히 훈련 세트에 과대적합되어있다. <br>
매개변수를 바꿔서 더 좋은 모델을 찾아야 한다. <br>
그 전에 검증 세트에 관해 좀 더 알아야 할 것이 있다.

### 교차 검증

검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할 수록 좋은 모델이 만들어진다. <br>
그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것이다. 

이럴 때 **교차 검증**을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.<br>
그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다. 

k-겹(폴드) 교차 검증은 훈련 세트를 k개의 부분으로 나눠 각 세트로 모델을 평가하는 것이다.

보통 5 / 10 차 교차 검증을 많이 사용한다. 이렇게 하면 데이터의 80 ~ 90%까지 훈련할 수 있다, <br>
검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.

사이킷런에는 cross_validate()라는 교차 검증 함수가 있다. 사용법은 간단하다.

1. 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달한다.
2. 그 다음 앞에서부터 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달한다.

사이킷런에는 cross_validate() 함수의 전신인 cross_val_score() 함수도 있다. <br>
이 함수는 cross_validate() 함수의 결과 중에서 test_score 값만 반환한다.

In [9]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00598383, 0.00698042, 0.00897455, 0.00598383, 0.00597954]), 'score_time': array([0.00199747, 0.00100994, 0.        , 0.00100112, 0.0010004 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. <br>
처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키마다 5개의 숫자가 담겨있다.<br>
cross_validate() 함수는 기본적으로 5-폴드 교차 검증을 수행한다. cv 매개변수에서 폴드 수를 바꿀 수도 있다.

훈련과 검증 시간은 리소스를 사용하는 상황에 따라 달라질 수 있으므로 <br>
fit_time , score_time 세트는 책과 실제 출력 결과가 다를 수 있다.

교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

In [10]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다.

한 가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다.

앞서 우리는 train_test_split 함수를 사용하여 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없었다.

하지만 만약 교차검증을 할 때 훈련 세트를 섞으려면 **분할기**를 사용해야 한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다. cross_validate() 함수는 기본적으로 회귀 모델일 경우 <br>
kFold 분할기를 사용하고, 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold 를 사용한다.

In [11]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))


0.855300214703487


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성한다,

In [12]:
# n_splits 매개변수 : 몇(k)폴드 교차검증을 할 지 정한다.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


kFold 클래스도 동일한 방식으로 사용할 수 있다. 

이어서 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아 보자. <br>
이때 테스트 세트를 사용하지 않고 교차 검증을 통해 좋은 모델을 고르면 된다.

### 하이퍼파라미터 튜닝

머신러닝 모델이 학습히는 파라미터를 **모델 파라마터**라고 한다 <br>
모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터 : **하이퍼파라미터**

사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현된다.

그럼 이런 하이퍼파라미터를 튜닝하는 작업은 어떻게 진행할까?

먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다. <br>
모델마다 적게는 1 - 2개에서 많게는 5 - 6개의 매개변수를 제공한다.